<a href="https://colab.research.google.com/github/limseo12/SideProject-4/blob/main/torch_hub_LJ_Speech/torch_hub_tacotron2_waveglow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy scipy librosa unidecode inflect

In [ ]:
import torch
tacotron2 = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tacotron2')
processing = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tts_utils')

waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')

In [ ]:
#디코더 부분에 프리넷과 포스트넷이 있는 것을 확인 할 수 있다.
#포스트넷 부분에 5개의 레이어로 구성 되어 있는 것을 확인 할 수 있다.
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

# waveglow 부분

In [ ]:
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

# Text To Speech(TTS)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')


def plot_data(data, figsize=(16, 4)):
  fig, axes = plt.subplots(1, len(data), figsize=figsize)
  for i in range(len(data)):
    axes[i].imshow(data[i], aspect='auto', origin='bottom',
                   interpolation='none', cmap='viridis')

def TTS(text):

  sampling_rate = 22050

  # sequence = np.array(tacotron2.text_to_sequence(text, ['english_cleaners']))[None, :]
  # sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.int64)
  sequence, input_lengths = processing.prepare_input_sequence([text]) 
  
  with torch.no_grad():
    # mel_outputs, mel_outputs_postnet, _, alignments = tacotron2.infer(sequence)
    mel_outputs_postnet, mel_lengths, alignments= tacotron2.infer(sequence, input_lengths) 
    audio = waveglow.infer(mel_outputs_postnet)

  # mel_output = mel_outputs.data.cpu().numpy()[0]
  mel_output_postnet = mel_outputs_postnet.data.cpu().numpy()[0]
  alignment = alignments.data.cpu().numpy()
  audio_np = audio[0].data.cpu().numpy()

  # return mel_output, mel_output_postnet, alignment, audio_np, sampling_rate
  return mel_output_postnet, alignment, audio_np, sampling_rate

In [ ]:
import librosa.display
from IPython.display import Audio

text = "Hello,  how are you?, My name is Lim,  Nice to meet you"
# mel_output, mel_output_postnet, alignment, audio_np, sampling_rate = TTS(text)
mel_output_postnet, alignment, audio_np, sampling_rate = TTS(text)

fig = plt.figure(figsize=(14, 4))
librosa.display.waveplot(audio_np, sr=sampling_rate)
# plot_data((mel_output, mel_output_postnet, alignment))
# plot_data((mel_output_postnet, alignment))
Audio(audio_np, rate=sampling_rate)